In [5]:
import json

from jury import Jury

In [3]:
mt_predictions = [
    ["the cat is on the mat", "There is cat playing on the mat"], 
    ["Look! a wonderful day."]
]
mt_references = [
    ["the cat is playing on the mat.", "The cat plays on the mat."],
    ["Today is a wonderful day", "The weather outside is wonderful."],
]

qa_predictions = ["1917", "Albert Einstein"]
qa_references = ["1917", "Einstein"]

MT_METRICS = ["bleu", "meteor", "rouge", "sacrebleu", "bertscore"]
QA_METRICS = ["squad"]

In [8]:
mt_jury = Jury(metrics=MT_METRICS, run_concurrent=True)
scores = mt_jury.evaluate(predictions=mt_predictions, references=mt_references)
print(json.dumps(scores, indent=4))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


ImportError: To be able to use this metric, you need to install the following dependencies['sacrebleu'] using 'pip install sacrebleu' for instance'